# Calculating Standardized Precipitation Evapotranspiration Index (SPEI)

This notebook demonstrates how to calculate the Standardized Precipitation Evapotranspiration Index (SPEI) using the `precip-index` package.

**What is SPEI?**
- Extension of SPI that includes temperature/evapotranspiration
- Developed by Vicente-Serrano et al. (2010)
- Uses water balance (P - PET) instead of precipitation alone
- More sensitive to temperature changes and global warming
- Better for agricultural and ecological drought assessment

**Learning Objectives:**
1. Calculate PET from temperature using Thornthwaite method
2. Compute SPEI with different input options
3. Compare SPI vs SPEI
4. Visualize and analyze SPEI results
5. Understand when to use SPEI vs SPI

## 1. Setup and Imports

In [ ]:
# Add src directory to Python path
import sys
sys.path.insert(0, '../src')

# Core libraries
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from datetime import datetime

# Import SPEI and SPI functions
from indices import (
    spi,
    spei, 
    spei_multi_scale,
    save_fitting_params,
    load_fitting_params,
    save_index_to_netcdf,
    classify_drought,
    get_drought_area_percentage
)

from utils import calculate_pet, eto_thornthwaite
from config import Periodicity

# Plotting settings
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ All imports successful!")

## 2. Generate Synthetic Climate Data

We'll create synthetic precipitation and temperature data for demonstration

In [ ]:
np.random.seed(42)

# Parameters
n_years = 30
n_months = n_years * 12
n_lat = 20
n_lon = 30
data_start_year = 1991

# Create coordinates
time = xr.cftime_range(
    start=f'{data_start_year}-01-01',
    periods=n_months,
    freq='MS',
    calendar='standard'
)

lat = np.linspace(-10, 10, n_lat)
lon = np.linspace(30, 50, n_lon)

# Generate precipitation (same as SPI notebook)
precip_data = np.zeros((n_months, n_lat, n_lon))

for t in range(n_months):
    month = t % 12
    seasonal_factor = 1 + 0.5 * np.sin(2 * np.pi * month / 12)
    
    for i in range(n_lat):
        for j in range(n_lon):
            spatial_factor = 0.5 + 0.5 * (i / n_lat + j / n_lon)
            mean_precip = 100 * seasonal_factor * spatial_factor
            alpha = 2.0
            beta = mean_precip / alpha
            
            precip_data[t, i, j] = np.random.gamma(alpha, beta)
            if np.random.rand() < 0.1:
                precip_data[t, i, j] = 0.0

precip = xr.DataArray(
    data=precip_data,
    dims=['time', 'lat', 'lon'],
    coords={'time': time, 'lat': lat, 'lon': lon},
    attrs={
        'long_name': 'Monthly precipitation',
        'units': 'mm/month',
        'standard_name': 'precipitation_amount'
    }
)

print("✓ Precipitation data created")
print(f"  Shape: {precip.shape}")
print(f"  Mean: {precip.mean().values:.1f} mm/month")

In [ ]:
# Generate synthetic temperature data
temp_data = np.zeros((n_months, n_lat, n_lon))

for t in range(n_months):
    month = t % 12
    year_idx = t // 12
    
    # Seasonal cycle (warmer in summer, cooler in winter)
    seasonal_temp = 15 * np.sin(2 * np.pi * (month - 3) / 12)  # Peak in June
    
    # Long-term warming trend (0.02°C/year)
    trend = 0.02 * year_idx
    
    for i in range(n_lat):
        for j in range(n_lon):
            # Latitudinal gradient (warmer at equator)
            base_temp = 25 - 0.5 * np.abs(lat[i])
            
            # Random variability
            noise = np.random.normal(0, 2)
            
            temp_data[t, i, j] = base_temp + seasonal_temp + trend + noise

temperature = xr.DataArray(
    data=temp_data,
    dims=['time', 'lat', 'lon'],
    coords={'time': time, 'lat': lat, 'lon': lon},
    attrs={
        'long_name': 'Monthly mean temperature',
        'units': 'degrees_Celsius',
        'standard_name': 'air_temperature'
    }
)

print("✓ Temperature data created")
print(f"  Shape: {temperature.shape}")
print(f"  Mean: {temperature.mean().values:.1f} °C")
print(f"  Range: [{temperature.min().values:.1f}, {temperature.max().values:.1f}] °C")

### Visualize Climate Data

In [ ]:
# Sample point
sample_lat_idx = n_lat // 2
sample_lon_idx = n_lon // 2

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))

# Precipitation time series
precip[:, sample_lat_idx, sample_lon_idx].plot(ax=ax1, linewidth=0.8, color='steelblue', label='Precipitation')
ax1.set_title(f'Precipitation (lat={lat[sample_lat_idx]:.1f}°, lon={lon[sample_lon_idx]:.1f}°)')
ax1.set_ylabel('Precipitation (mm/month)')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Temperature time series with trend
temp_series = temperature[:, sample_lat_idx, sample_lon_idx]
temp_series.plot(ax=ax2, linewidth=0.8, color='orangered', label='Temperature')

# Add trend line
x = np.arange(len(temp_series))
z = np.polyfit(x, temp_series.values, 1)
p = np.poly1d(z)
ax2.plot(temp_series.time, p(x), 'k--', linewidth=1.5, alpha=0.7, label=f'Trend: +{z[0]*12:.2f}°C/decade')

ax2.set_title(f'Temperature with Warming Trend (lat={lat[sample_lat_idx]:.1f}°, lon={lon[sample_lon_idx]:.1f}°)')
ax2.set_ylabel('Temperature (°C)')
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

## 3. Calculate PET using Thornthwaite Method

PET (Potential Evapotranspiration) can be calculated from temperature and latitude

In [ ]:
print("Calculating PET using Thornthwaite method...")

# Calculate PET
pet = calculate_pet(
    temperature=temperature,
    latitude=lat,  # Can be array or single value
    data_start_year=data_start_year
)

print("✓ PET calculation complete!")
print(f"  Shape: {pet.shape}")
print(f"  Mean: {pet.mean().values:.1f} mm/month")
print(f"  Range: [{pet.min().values:.1f}, {pet.max().values:.1f}] mm/month")

### Visualize Water Balance Components

In [ ]:
# Calculate water balance
water_balance = precip - pet

fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Precipitation
precip[:, sample_lat_idx, sample_lon_idx].plot(ax=axes[0], linewidth=0.8, color='steelblue')
axes[0].set_title('Precipitation (P)')
axes[0].set_ylabel('P (mm/month)')
axes[0].grid(True, alpha=0.3)

# PET
pet[:, sample_lat_idx, sample_lon_idx].plot(ax=axes[1], linewidth=0.8, color='orangered')
axes[1].set_title('Potential Evapotranspiration (PET)')
axes[1].set_ylabel('PET (mm/month)')
axes[1].grid(True, alpha=0.3)

# Water Balance (P - PET)
wb = water_balance[:, sample_lat_idx, sample_lon_idx]
wb.plot(ax=axes[2], linewidth=0.8, color='darkgreen')
axes[2].axhline(y=0, color='k', linestyle='-', linewidth=0.8, alpha=0.3)
axes[2].fill_between(wb.time, 0, wb, where=(wb >= 0), alpha=0.3, color='blue', label='Surplus')
axes[2].fill_between(wb.time, 0, wb, where=(wb < 0), alpha=0.3, color='red', label='Deficit')
axes[2].set_title('Water Balance (P - PET)')
axes[2].set_ylabel('P - PET (mm/month)')
axes[2].grid(True, alpha=0.3)
axes[2].legend()

plt.suptitle(f'Water Balance Components (lat={lat[sample_lat_idx]:.1f}°, lon={lon[sample_lon_idx]:.1f}°)', 
             fontsize=14, y=0.995)
plt.tight_layout()
plt.show()

# Statistics
print(f"\nWater Balance Statistics:")
print(f"  Mean P: {precip.mean().values:.1f} mm/month")
print(f"  Mean PET: {pet.mean().values:.1f} mm/month")
print(f"  Mean (P-PET): {water_balance.mean().values:.1f} mm/month")
print(f"  Deficit months: {(water_balance < 0).sum().values} ({100*(water_balance < 0).sum()/water_balance.size:.1f}%)")

## 4. Calculate SPEI

### Option 1: Using Pre-computed PET

In [ ]:
print("Calculating SPEI-12 with pre-computed PET...")

spei_12, params = spei(
    precip=precip,
    pet=pet,
    scale=12,
    periodicity='monthly',
    calibration_start_year=1991,
    calibration_end_year=2020,
    return_params=True
)

print("✓ SPEI-12 calculation complete!")
print(f"  Output shape: {spei_12.shape}")
print(f"  Valid range: [{spei_12.min().values:.2f}, {spei_12.max().values:.2f}]")
print(f"  Mean: {spei_12.mean().values:.3f}")
print(f"  Std: {spei_12.std().values:.3f}")

### Option 2: Calculate PET from Temperature (Automatic)

In [ ]:
print("Calculating SPEI-12 from temperature (auto PET calculation)...")

spei_12_auto = spei(
    precip=precip,
    temperature=temperature,
    latitude=lat,
    scale=12,
    periodicity='monthly',
    calibration_start_year=1991,
    calibration_end_year=2020
)

print("✓ SPEI-12 calculation complete!")

# Verify results are identical
diff = np.abs(spei_12.values - spei_12_auto.values)
print(f"  Max difference from manual PET: {np.nanmax(diff):.10f}")
print("  Results are identical!" if np.nanmax(diff) < 1e-6 else "  Warning: Results differ!")

## 5. Compare SPI vs SPEI

Let's calculate SPI-12 and compare with SPEI-12

In [ ]:
print("Calculating SPI-12 for comparison...")

spi_12 = spi(
    precip=precip,
    scale=12,
    periodicity='monthly',
    calibration_start_year=1991,
    calibration_end_year=2020
)

print("✓ SPI-12 calculation complete!")

In [ ]:
# Compare SPI and SPEI
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# SPI
spi_series = spi_12[:, sample_lat_idx, sample_lon_idx]
spi_series.plot(ax=axes[0], linewidth=0.8, color='steelblue', label='SPI-12')
axes[0].axhline(y=0, color='k', linestyle='-', linewidth=0.8, alpha=0.3)
axes[0].axhline(y=-1.0, color='orange', linestyle='--', linewidth=0.6, alpha=0.4)
axes[0].axhline(y=-2.0, color='red', linestyle='--', linewidth=0.6, alpha=0.4)
axes[0].fill_between(spi_series.time, -10, 0, alpha=0.1, color='red')
axes[0].fill_between(spi_series.time, 0, 10, alpha=0.1, color='blue')
axes[0].set_title('SPI-12 (Precipitation only)')
axes[0].set_ylabel('SPI-12')
axes[0].set_ylim(-3, 3)
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# SPEI
spei_series = spei_12[:, sample_lat_idx, sample_lon_idx]
spei_series.plot(ax=axes[1], linewidth=0.8, color='darkgreen', label='SPEI-12')
axes[1].axhline(y=0, color='k', linestyle='-', linewidth=0.8, alpha=0.3)
axes[1].axhline(y=-1.0, color='orange', linestyle='--', linewidth=0.6, alpha=0.4)
axes[1].axhline(y=-2.0, color='red', linestyle='--', linewidth=0.6, alpha=0.4)
axes[1].fill_between(spei_series.time, -10, 0, alpha=0.1, color='red')
axes[1].fill_between(spei_series.time, 0, 10, alpha=0.1, color='blue')
axes[1].set_title('SPEI-12 (Precipitation - PET)')
axes[1].set_ylabel('SPEI-12')
axes[1].set_ylim(-3, 3)
axes[1].grid(True, alpha=0.3)
axes[1].legend()

# Comparison (both on same plot)
spi_series.plot(ax=axes[2], linewidth=1.0, color='steelblue', label='SPI-12', alpha=0.7)
spei_series.plot(ax=axes[2], linewidth=1.0, color='darkgreen', label='SPEI-12', alpha=0.7)
axes[2].axhline(y=0, color='k', linestyle='-', linewidth=0.8, alpha=0.3)
axes[2].axhline(y=-1.0, color='orange', linestyle='--', linewidth=0.6, alpha=0.4)
axes[2].set_title('SPI vs SPEI Comparison')
axes[2].set_ylabel('Index Value')
axes[2].set_ylim(-3, 3)
axes[2].grid(True, alpha=0.3)
axes[2].legend()

plt.suptitle(f'SPI vs SPEI (lat={lat[sample_lat_idx]:.1f}°, lon={lon[sample_lon_idx]:.1f}°)', 
             fontsize=14, y=0.995)
plt.tight_layout()
plt.show()

# Calculate correlation
valid_mask = ~(np.isnan(spi_series.values) | np.isnan(spei_series.values))
correlation = np.corrcoef(spi_series.values[valid_mask], spei_series.values[valid_mask])[0, 1]
print(f"\nCorrelation between SPI and SPEI: {correlation:.3f}")

# Count drought months
spi_drought = (spi_series <= -1.0).sum().values
spei_drought = (spei_series <= -1.0).sum().values
print(f"\nDrought months (index ≤ -1.0):")
print(f"  SPI-12: {spi_drought} months ({100*spi_drought/len(spi_series):.1f}%)")
print(f"  SPEI-12: {spei_drought} months ({100*spei_drought/len(spei_series):.1f}%)")
print(f"  Difference: {spei_drought - spi_drought} months")

### Key Differences: SPI vs SPEI

**SPI (Precipitation only):**
- ✓ Simple, requires only precipitation data
- ✓ Good for meteorological drought
- ✗ Doesn't account for temperature/evaporation
- ✗ May miss agricultural drought in warming climates

**SPEI (Precipitation - PET):**
- ✓ Accounts for temperature and evaporation
- ✓ Better for agricultural and ecological drought
- ✓ More sensitive to climate change
- ✗ Requires temperature data or PET
- ✗ Slightly more complex calculation

**When to use SPEI:**
- Agricultural applications
- Vegetation/ecosystem monitoring
- Climate change impact studies
- Regions with significant temperature trends

## 6. Multi-Scale SPEI Analysis

In [ ]:
print("Calculating SPEI for scales: 1, 3, 6, 12 months...")

spei_multi = spei_multi_scale(
    precip=precip,
    pet=pet,
    scales=[1, 3, 6, 12],
    periodicity='monthly',
    calibration_start_year=1991,
    calibration_end_year=2020
)

print("✓ Multi-scale SPEI calculation complete!")
print(f"  Variables: {list(spei_multi.data_vars)}")

In [ ]:
# Compare different SPEI time scales
fig, axes = plt.subplots(4, 1, figsize=(14, 12))

scales = [1, 3, 6, 12]
colors = ['lightgreen', 'mediumseagreen', 'seagreen', 'darkgreen']

for i, (scale, color) in enumerate(zip(scales, colors)):
    var_name = f'spei_gamma_{scale}_month'
    spei_data = spei_multi[var_name]
    
    spei_data[:, sample_lat_idx, sample_lon_idx].plot(
        ax=axes[i], 
        linewidth=0.8, 
        color=color
    )
    
    axes[i].axhline(y=0, color='k', linestyle='-', linewidth=0.8, alpha=0.3)
    axes[i].axhline(y=-1.0, color='orange', linestyle='--', linewidth=0.6, alpha=0.4)
    axes[i].axhline(y=-2.0, color='red', linestyle='--', linewidth=0.6, alpha=0.4)
    axes[i].fill_between(spei_data.time, -10, 0, alpha=0.1, color='red')
    axes[i].fill_between(spei_data.time, 0, 10, alpha=0.1, color='blue')
    
    axes[i].set_title(f'SPEI-{scale}')
    axes[i].set_ylabel(f'SPEI-{scale}')
    axes[i].set_ylim(-3, 3)
    axes[i].grid(True, alpha=0.3)
    
    if i < 3:
        axes[i].set_xlabel('')

plt.suptitle(f'SPEI Time Series Comparison (lat={lat[sample_lat_idx]:.1f}°, lon={lon[sample_lon_idx]:.1f}°)', 
             fontsize=14, y=0.995)
plt.tight_layout()
plt.show()

## 7. Spatial Analysis

In [ ]:
# Create spatial maps for different time periods
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

# Select 4 different time points
time_indices = [-36, -24, -12, -1]  # Last 3, 2, 1 years, and current
titles = ['3 years ago', '2 years ago', '1 year ago', 'Current']

for i, (t_idx, title) in enumerate(zip(time_indices, titles)):
    im = axes[i].pcolormesh(
        spei_12.lon,
        spei_12.lat,
        spei_12[t_idx, :, :],
        cmap='RdYlBu',
        vmin=-2.5,
        vmax=2.5,
        shading='auto'
    )
    axes[i].set_title(f'{title} ({spei_12.time[t_idx].values})')
    axes[i].set_xlabel('Longitude')
    axes[i].set_ylabel('Latitude')
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=axes[i])
    cbar.set_label('SPEI-12')

plt.suptitle('SPEI-12 Spatial Patterns Over Time', fontsize=14, y=0.995)
plt.tight_layout()
plt.show()

## 8. Drought Area Percentage Analysis

In [ ]:
# Calculate drought area for both SPI and SPEI
spi_drought_area = get_drought_area_percentage(spi_12, threshold=-1.0)
spei_drought_area = get_drought_area_percentage(spei_12, threshold=-1.0)

# Plot comparison
fig, ax = plt.subplots(figsize=(14, 5))

spi_drought_area.plot(ax=ax, linewidth=1.2, color='steelblue', label='SPI-12', alpha=0.7)
spei_drought_area.plot(ax=ax, linewidth=1.2, color='darkgreen', label='SPEI-12', alpha=0.7)

ax.axhline(y=spi_drought_area.mean(), color='steelblue', linestyle='--', 
           linewidth=0.8, alpha=0.5, label=f'SPI mean: {spi_drought_area.mean().values:.1f}%')
ax.axhline(y=spei_drought_area.mean(), color='darkgreen', linestyle='--', 
           linewidth=0.8, alpha=0.5, label=f'SPEI mean: {spei_drought_area.mean().values:.1f}%')

ax.set_title('Drought Area Percentage: SPI vs SPEI (Index ≤ -1.0)', fontsize=12)
ax.set_ylabel('Area under drought (%)')
ax.set_ylim(0, 100)
ax.grid(True, alpha=0.3)
ax.legend()

plt.tight_layout()
plt.show()

print(f"\nDrought Area Statistics:")
print(f"  SPI-12 mean: {spi_drought_area.mean().values:.1f}%")
print(f"  SPEI-12 mean: {spei_drought_area.mean().values:.1f}%")
print(f"  Difference: {(spei_drought_area.mean() - spi_drought_area.mean()).values:.1f}%")

## 9. Save Results

In [ ]:
# Save SPEI results
save_index_to_netcdf(spei_12, 'spei_12_output.nc', compress=True, complevel=5)
print("✓ SPEI-12 saved to: spei_12_output.nc")

# Save multi-scale SPEI
save_index_to_netcdf(spei_multi, 'spei_multi_scale_output.nc', compress=True, complevel=5)
print("✓ Multi-scale SPEI saved to: spei_multi_scale_output.nc")

# Save fitting parameters
save_fitting_params(
    params,
    'spei_gamma_params_12_month.nc',
    scale=12,
    periodicity='monthly',
    index_type='spei',
    calibration_start_year=1991,
    calibration_end_year=2020,
    coords={'lat': precip.lat, 'lon': precip.lon}
)
print("✓ SPEI parameters saved to: spei_gamma_params_12_month.nc")

# Save PET for future use
pet.to_netcdf('pet_thornthwaite.nc')
print("✓ PET saved to: pet_thornthwaite.nc")

## 10. Summary and Best Practices

### Key Takeaways:

1. **SPEI vs SPI:**
   - SPEI includes temperature effects through PET
   - SPEI generally shows more drought events in warming climates
   - Use SPEI for agricultural/ecological applications
   - Use SPI when only precipitation data available

2. **PET Calculation:**
   - Thornthwaite method requires only temperature and latitude
   - Can be calculated automatically within SPEI function
   - Can also provide pre-computed PET from other methods

3. **Interpretation:**
   - SPEI values interpreted same as SPI (standard normal distribution)
   - Negative values indicate drought conditions
   - SPEI typically shows stronger trends in warming climates

### Recommended Workflow for SPEI:

```python
# 1. Load climate data
ds = xr.open_dataset('climate_data.nc')
precip = ds['precip']
temp = ds['temperature']
lat = ds['lat']

# 2. Calculate SPEI (automatic PET calculation)
spei_12, params = spei(
    precip, 
    temperature=temp,
    latitude=lat,
    scale=12, 
    return_params=True
)

# 3. Save parameters for reuse
save_fitting_params(params, 'spei_params.nc', scale=12, periodicity='monthly')

# 4. Analyze results
drought_area = get_drought_area_percentage(spei_12, threshold=-1.0)
```

### Applications:
- 🌾 Agricultural drought monitoring
- 🌳 Forest fire risk assessment
- 💧 Water resource management
- 🌡️ Climate change impact studies
- 🗺️ Regional drought comparison

### Next Steps:
- Apply to your own climate datasets
- Experiment with different time scales
- Compare with observed drought impacts
- Integrate with crop models or ecological models